# **Game Management**

In this notebook, we will implement the game management system, including communication with the game and all associated mechanisms. The Python code handles the computer vision aspect of the project, implementing:

- Face Registration and Recognition, used for users' signup and login.
- Emotion Recognition, to adjust the difficulty of the game based on the player's emotions.
- Body Tracking, to control the virtual character.

## **Face Registration (User Signup)**

This section involves recording a 10-second video from the webcam. The frames composing the video will be processed to identify a feasible one in which a face is detected. This process utilizes dlib's face detector, known for its high precision and accuracy. The frames deemed valid will be stored in the UserFaces directory.

### Video Acquisition

This Python script captures video from a webcam, saves it to a file, and performs face detection using OpenCV and dlib. The threaded design sets a stop flag after 10 seconds to gracefully terminate the video capture loop. The recorded video frames are stored in a directory named after the provided username. Face detection is executed on the captured video, and the temporary file is removed.

In [1]:
import cv2
import threading
import time
import os
import dlib
import numpy as np

# Thread-safe variable
stop_thread = False
lock = threading.Lock()

def set_stop_thread():

    """
    Thread function to set the stop_thread variable to True after 10 seconds.
    """
    
    global stop_thread
    time.sleep(10)  # Wait for 10 seconds
    with lock:
        stop_thread = True

def video_acquisition(username: str):

    """
    Capture video from the webcam, save it to a file, and perform face detection.

    Parameters:
    - username: User identifier for creating output folder

    Returns:
    - True if faces are detected, False otherwise
    """

    # Start the thread that sets the stop_thread variable after 10 seconds
    stop_thread_thread = threading.Thread(target=set_stop_thread)
    stop_thread_thread.start()

    # Ensure the directory exists for storing face images
    output_directory = f'../UserFaces/{username}/'
    os.makedirs(output_directory, exist_ok=True)
    video_path = os.path.join(output_directory, 'video.avi') # Define the path for storing the captured video

    capture = cv2.VideoCapture(0) # Open a connection to the webcam

    # Define the codec and create a VideoWriter object to save the video
    fourcc = cv2.VideoWriter_fourcc('X', 'V', 'I', 'D')
    videoWriter = cv2.VideoWriter(video_path, fourcc, 30.0, (640, 480), True)

    while True:
        ret, frame = capture.read() # Read a frame from the webcam

        if ret:
            cv2.imshow('Webcam', frame)  # Display the frame from the webcam
            videoWriter.write(frame) # Write the frame to the video file

        with lock:
            # Check if the stop_thread variable is set to True
            if stop_thread:
                break
        
        if cv2.waitKey(1) == 27:
            break

    # Release the capture and videoWriter objects
    capture.release()
    videoWriter.release()

    cv2.destroyAllWindows() # Close the OpenCV window
    stop_thread_thread.join() # Wait for the stop_thread thread to finish

### Frame Proccesing

The process_frame function detects faces in a single input frame using the dlib library, returning True if faces are found and False otherwise. The detect_faces function processes a video, saves frames with detected faces, and returns True if a sufficient number of faces are detected across frames, otherwise False. Using these functions we can simply create a 'database' for each user.

In [2]:
def process_frame(frame):

    """
    Process a single frame to detect faces.

    Parameters:
    - frame: Input frame (image)

    Returns:
    - True if faces are detected, False otherwise
    """
    
    face_detector = dlib.get_frontal_face_detector() # Initialize the face detector using dlib
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert the frame to grayscale
    faces = face_detector(gray, 1) # Detect faces in the grayscale image

    if faces: return True  # Faces detected

    return False  # No faces detected

def detect_faces(username: str):

    """
    Process a video to detect and save faces in frames.

    Parameters:
    - username: User identifier used for output folder

    Returns:
    - True if faces are detected in enough frames, False otherwise
    """

    output_directory = os.path.join('..', 'UserFaces', username) # Set the output directory for storing face images
    cap = cv2.VideoCapture(os.path.join(output_directory, 'video.avi')) # Open the video file for processing

    # Check if the video file is opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    last_valid_frame = None  # Index of the last valid frame
    valid_frames = 0 # Number of valid frames
    frame_count = 0
    total_num_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Loop through each frame in the video
    while True:

        ret, frame = cap.read() # Read the next frame from the video

        # Check if the end of the video is reached
        if not ret:
            print("End of video.")
            break

        # Calculate the percentage completion
        percentage_completion = (frame_count / total_num_of_frames) * 100
        if frame_count % 50 == 0:
            print(f"Progress: {int(percentage_completion)}%")
            game_communicator.send_to_game(f'{int(percentage_completion)}')

        # Skip frames if they are too close to the previous valid frame
        if (last_valid_frame and frame_count // 15 == last_valid_frame // 15):
            frame_count += 1
            continue

        # Process the frame to detect faces
        if process_frame(frame):
            # Save the face in the output folder
            last_valid_frame = frame_count
            valid_frames += 1
            cv2.imwrite(os.path.join(output_directory, f'face_{valid_frames}.png'), frame)

        frame_count += 1  # Increment the frame count

    cap.release() # Release the video capture object

    print(f'{valid_frames} Valid frames found!')

    # Check if an insufficient number of valid frames are detected
    if valid_frames < 10:
        return False  # Not enough valid frames

    return True  # Faces detected in enough frames


### Face Registration

Through this function we can put all the pieces toghether, hence acquire the video and save the detected faces into the user's directory.

In [3]:
def face_registration(username: str):
    
    """
    Main function for face registration.

    Parameters:
    - username: User identifier for creating output folder

    Returns:
    - Result of face detection (True if faces are detected, False otherwise)
    """

    video_acquisition(username)

    # Perform face detection on the captured video
    result = detect_faces(username)

    # Remove the temporary video file
    os.remove(os.path.join('..', 'UserFaces', username, 'video.avi'))

    return result

## **Face Recognition (User Login)**

For the face recognition part, we will import from the Modules the face_recognition function. Note that the 'face_recognition.py' is the Notebook 'face_recognition.ipynb' converted into a Python file. 

In [4]:
from Modules.face_recognition import face_recognition 
from Modules.face_recognition import learn_faces 

## **Game Communication**

In [5]:
from Modules.game_communicator import GameCommunicator

face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor("../Models/shape_predictor_68_face_landmarks.dat")
face_encoder = dlib.face_recognition_model_v1("../Models/dlib_face_recognition_resnet_model_v1.dat")

known_faces = learn_faces(face_detector, shape_predictor, face_encoder)

game_communicator = GameCommunicator()
game_communicator.wait_for_connection()

while True:

    msg = game_communicator.receive_from_game()
    msg = msg.split(':')

    if msg[0] == "face_registration":
        username = msg[1]
        if(face_registration(username)):
            game_communicator.send_to_game("success")
        else:
            game_communicator.send_to_game("fail")

    if msg[0] == "face_recognition":

        username = face_recognition(known_faces, face_detector, shape_predictor, face_encoder)
        game_communicator.send_to_game(username)

NameError: name 'learn_faces' is not defined